In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tc import ThermoCycler
tc = ThermoCycler(ip="169.254.151.99")
# res = tc.open_door()

In [82]:
# Example usage with PreMethod and Method dataclass
from datetime import datetime

import xml
from tc import MetaData, Method, MethodStep, PIDMember, PreMethod, _serialize_dataclass_to_xml, dataclass_to_xml
import xml.etree.ElementTree as ET


metadata = MetaData(method_name="method 1", creator="inheco")
pre_method = PreMethod(target_block_temperature=20, target_lid_temperature=110, metadata=metadata)

method_steps = [
    MethodStep(number=1, slope=4.4, plateau_temperature=95, plateau_time=60, over_shoot_slope_1=4.4, over_shoot_temperature=5, over_shoot_time=0, over_shoot_slope_2=2.2, goto_number=0, loop_number=0, pid_number=1, lid_temp=110),
    # Add more MethodStep instances as needed
]

pid_members = [
    PIDMember(number="1", p_heating=60, p_cooling=80, i_heating=250, i_cooling=100, d_heating=10, d_cooling=10, p_lid=100, i_lid=70)
]

method = Method(variant=960000, plate_type=0, fluid_quantity=0, post_heating=False, start_block_temperature=20, start_lid_temperature=110, steps=method_steps, pid_set=pid_members)

# Serialize to XML
# asdf = dataclass_to_xml(pre_method, "PreMethod", metadata)
asdf = dataclass_to_xml(method, "Method")
# asdf = ET.tostring(pre_method.to_xml())


# Output the XML strings
dom = xml.dom.minidom.parseString(asdf)
pretty_xml = dom.toprettyxml(indent="  ")  # Using 2 spaces for indentation
print( pretty_xml)
# print(method_xml)

# print(metadata.to_dict())


<?xml version="1.0" ?>
<PreMethod methodName="method 1" creator="inheco" description="" dateTime="2024-09-30T15:06:58.076708">
  <TargetBlockTemperature>20</TargetBlockTemperature>
  <TargetLidTemperature>110</TargetLidTemperature>
</PreMethod>



In [10]:
await tc.stop()

<Response [200]>

In [4]:
res = await tc.close_door()
res.status_code

200

In [ ]:
# http && !(http contains "GetStatus") && !(http contains "Continue")


In [ ]:
import xml.etree.ElementTree as ET
import time
namespaces = {
    'SOAP-ENV': 'http://schemas.xmlsoap.org/soap/envelope/',
    's': 'http://sila.coop'
}

for i in range(30):
  print("wait ",i)
  time.sleep(1)
  a =tc.get_status1()
  root = ET.fromstring(a.text)
  print(root.find('.//s:state', namespaces).text)

In [9]:
await tc.get_state()

<DeviceState.BUSY: 'busy'>

In [ ]:
tc.close_door()

In [ ]:
import requests
# POST / HTTP/1.1
# Content-Type: text/xml; charset=utf-8
# SOAPAction: "http://sila.coop/CloseDoor"
# Host: 169.254.245.237:8080
# Content-Length: 249
# Expect: 100-continue

requests.post("http://10.8.8.1:8080/",
              data='<s:Envelope xmlns:s="http://schemas.xmlsoap.org/soap/envelope/"><s:Body><OpenDoor xmlns="http://sila.coop" xmlns:i="http://www.w3.org/2001/XMLSchema-instance"><requestId>1579489236</requestId><lockId i:nil="true"/></OpenDoor></s:Body></s:Envelope>',
              headers={"Content-Type": "text/xml; charset=utf-8",
                        "SOAPAction": "http://sila.coop/OpenDoor",
                        "Host": "10.8.8.1:8080",
              }, timeout=1).text